In [11]:
from pyspark import SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
import math
import pyspark
import os
import math
import pandas as pd

os.environ["PYSPARK_PYTHON3"] = "python"
spark_home = os.environ.get('SPARK_HOME', None)
print (spark_home)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel('ERROR')
sc

# Step 2: Load Data into RDD
data_rdd = sc.textFile("dataset/filtered50movies.csv")

header = data_rdd.first()
data_rdd = data_rdd.filter(lambda line: line != header)

# Mapping the data to a tuple (UserID, (MovieID, Rating))
user_movie_ratings = data_rdd.map(lambda line: line.split(',')).map(lambda tokens: (int(tokens[0]), (int(tokens[1]), float(tokens[2]))))

# Group by UserID
grouped_data = user_movie_ratings.groupByKey().mapValues(list)

# Calculate the number of unique movies
unique_movies_rdd = user_movie_ratings.map(lambda x: x[1][0]).distinct()
num_movies = unique_movies_rdd.count()
print(num_movies)

# Step 3: Data Preprocessing
# Convert grouped data to sparse vectors (UserID, SparseVector)
def to_sparse_vector(ratings, num_movies):
    indices = [item[0] for item in ratings]
    values = [item[1] for item in ratings]
    return Vectors.sparse(num_movies, indices, values)

user_vectors = grouped_data.map(lambda user_ratings: (user_ratings[0], to_sparse_vector(user_ratings[1], num_movies)))
print(user_vectors.collect())

# Step 4: Calculate Pairwise Cosine Distance
def cosine_similarity(vector1, vector2):
    return vector1.dot(vector2) / (math.sqrt(vector1.dot(vector1)) * math.sqrt(vector2.dot(vector2)))

pairs = user_vectors.cartesian(user_vectors)

# Compute cosine similarity for each pair, filter out pairs of the same user
cosine_distances = pairs.filter(lambda pair: pair[0][0] != pair[1][0]).map(lambda pair: ((pair[0][0], pair[1][0]), cosine_similarity(pair[0][1], pair[1][1])))

# Collect the results
cosine_distances = cosine_distances.collect()


None
42
[(18, SparseVector(42, {1: 1.0, 6: 1.5, 47: 2.0, 50: 2.5, 70: 1.0, 110: 2.0, 163: 1.5, 223: 1.5, 231: -0.5, 235: 2.0, 260: 1.5, 296: 1.5, 356: 2.0, 367: -0.5, 457: 1.0, 480: 1.0, 500: 1.0, 527: 2.0, 590: 1.5, 593: 2.0, 608: 2.0, 648: 0.5, 673: 0.5, 780: 1.5, 919: 1.5, 923: 2.5})), (32, SparseVector(42, {1: 0.5, 3: 0.5, 6: 0.5, 50: 2.5, 260: 2.5, 296: 2.5, 527: 1.5, 553: 0.5, 592: 2.5, 593: 2.5, 608: 2.5, 648: 0.5, 661: 0.5, 733: 1.5, 736: 0.5, 780: 0.5})), (40, SparseVector(42, {1: 2.5, 110: 1.5, 151: 1.5, 163: 2.5, 216: -2.0, 223: 2.5, 231: -2.0, 235: 1.5, 296: 2.5, 316: 0.5, 362: 1.5, 367: 1.5, 457: 0.5, 553: 2.5, 590: 1.5, 592: 1.5})), (44, SparseVector(42, {1: 0.5, 3: 0.5, 6: 0.5, 260: 2.5, 648: 0.5, 661: 0.5, 673: -2.0, 733: 2.5, 736: -1.0, 780: 0.5})), (46, SparseVector(42, {1: 2.5, 50: 2.5, 110: 2.5, 231: 0.5, 316: 1.5, 349: 0.5, 367: 1.5, 457: 2.5, 590: 2.5, 592: 0.5, 593: 2.5})), (50, SparseVector(42, {1: 0.5, 296: 1.5, 356: 0.5, 593: 1.0, 596: -1.0, 608: 1.5, 923: 1.5

23/11/12 00:12:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar